In [ ]:
!pip install catboost

In [ ]:
import datetime
import gc
import sys
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import catboost
from catboost import Pool
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
train_monthly = pd.read_csv('drive/MyDrive/Colab Notebooks/kaggle/PredictFutureSales/data/dataset_v1.csv', index_col=0)
sys.getsizeof(train_monthly)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


1345531024

In [ ]:
train_monthly

,date_block_num,shop_id,item_id,item_category_id,item_price,mean_item_price,item_cnt,mean_item_cnt,transactions,year,month,item_cnt_nextmonth,item_price_unit,hist_min_item_price,hist_max_item_price,price_increase,item_cnt_min,item_cnt_max,item_cnt_mean,item_cnt_std,item_cnt_shifted1,item_cnt_shifted2,item_cnt_shifted3,item_trend
0,0,2,5572,2.0,10730.00,1532.8572,9.0,1.285714,7.0,2013,0,1.0,1192.0,0.0,18979.5,8249.50,9.0,9.0,9.0,0.0,0.0,0.0,0.0,2.25
1,0,2,5643,2.0,4775.21,2387.6050,0.0,0.000000,2.0,2013,0,0.0,inf,0.0,35260.0,30484.79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
2,0,2,5583,5.0,1188.30,594.1500,2.0,1.000000,2.0,2013,0,1.0,594.0,0.0,5592.0,4403.70,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.50
3,0,2,7893,6.0,5970.00,1990.0000,3.0,1.000000,3.0,2013,0,2.0,1990.0,0.0,27950.0,21980.00,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.75
4,0,2,7894,6.0,1490.00,1490.0000,1.0,1.000000,1.0,2013,0,2.0,1490.0,0.0,25880.0,24390.00,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6727650,33,36,9103,0.0,0.00,0.0000,0.0,0.000000,0.0,2015,9,NaN,0.0,0.0,500.0,500.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
6727651,33,36,9107,0.0,0.00,0.0000,0.0,0.000000,0.0,2015,9,NaN,0.0,0.0,300.0,300.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
6727652,33,36,5704,0.0,0.00,0.0000,0.0,0.000000,0.0,2015,9,NaN,0.0,0.0,1750.0,1750.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
6727653,33,36,12733,0.0,0.00,0.0000,0.0,0.000000,0.0,2015,9,NaN,0.0,0.0,2980.0,2980.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00


Train/validation split

In [ ]:
train_set = train_monthly.query('date_block_num>=3 and date_block_num<28').copy()
validation_set = train_monthly.query('date_block_num>=28 and date_block_num<33').copy()
test_set = train_monthly.query('date_block_num==33').copy()

train_set.dropna(inplace=True)
validation_set.dropna(inplace=True)

print('Train set records:', train_set.shape[0])
print('Validation set records:', validation_set.shape[0])
print('Test set records:', test_set.shape[0])

print('Train set records: %s (%.f%% of complete data)' % (train_set.shape[0], ((train_set.shape[0]/train_monthly.shape[0])*100)))
print('Validation set records: %s (%.f%% of complete data)' % (validation_set.shape[0], ((validation_set.shape[0]/train_monthly.shape[0])*100)))



Train set records: 4946416
Validation set records: 989350
Test set records: 197867
Train set records: 4946416 (74% of complete data)
Validation set records: 989350 (15% of complete data)


In [ ]:
del train_monthly

In [ ]:
train_set

,date_block_num,shop_id,item_id,item_category_id,item_price,mean_item_price,item_cnt,mean_item_cnt,transactions,year,month,item_cnt_nextmonth,item_price_unit,hist_min_item_price,hist_max_item_price,price_increase,item_cnt_min,item_cnt_max,item_cnt_mean,item_cnt_std,item_cnt_shifted1,item_cnt_shifted2,item_cnt_shifted3,item_trend
593820,3,2,5572,2.0,2980.0,1490.0,2.0,1.0,2.0,2013,3,2.0,1490.0,0.0,18979.5,15999.5,1.0,2.0,1.333333,0.577350,1.0,1.0,9.0,-2.25
593821,3,2,5643,0.0,0.0,0.0,0.0,0.0,0.0,2013,3,5.0,0.0,0.0,35260.0,35260.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00
593822,3,2,5583,0.0,0.0,0.0,0.0,0.0,0.0,2013,3,0.0,0.0,0.0,5592.0,5592.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00
593823,3,2,7893,6.0,9350.0,1870.0,5.0,1.0,5.0,2013,3,1.0,1870.0,0.0,27950.0,18600.0,1.0,5.0,2.666667,2.081666,1.0,2.0,3.0,-0.25
593824,3,2,7894,0.0,0.0,0.0,0.0,0.0,0.0,2013,3,0.0,0.0,0.0,25880.0,25880.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5540260,27,36,9103,0.0,0.0,0.0,0.0,0.0,0.0,2015,3,0.0,0.0,0.0,500.0,500.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00
5540261,27,36,9107,0.0,0.0,0.0,0.0,0.0,0.0,2015,3,0.0,0.0,0.0,300.0,300.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00
5540262,27,36,5704,0.0,0.0,0.0,0.0,0.0,0.0,2015,3,0.0,0.0,0.0,1750.0,1750.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00
5540263,27,36,12733,0.0,0.0,0.0,0.0,0.0,0.0,2015,3,0.0,0.0,0.0,2980.0,2980.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00


In [ ]:
validation_set

,date_block_num,shop_id,item_id,item_category_id,item_price,mean_item_price,item_cnt,mean_item_cnt,transactions,year,month,item_cnt_nextmonth,item_price_unit,hist_min_item_price,hist_max_item_price,price_increase,item_cnt_min,item_cnt_max,item_cnt_mean,item_cnt_std,item_cnt_shifted1,item_cnt_shifted2,item_cnt_shifted3,item_trend
5540265,28,2,5572,2.0,1590.0,1590.00,1.0,1.0,1.0,2015,4,0.0,1590.0,0.0,18979.5,17389.5,1.0,1.0,1.000000,0.000000,1.0,1.0,1.0,-0.5
5540266,28,2,5643,0.0,0.0,0.00,0.0,0.0,0.0,2015,4,0.0,0.0,0.0,35260.0,35260.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
5540267,28,2,5583,0.0,0.0,0.00,0.0,0.0,0.0,2015,4,0.0,0.0,0.0,5592.0,5592.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
5540268,28,2,7893,0.0,0.0,0.00,0.0,0.0,0.0,2015,4,4.0,0.0,0.0,27950.0,27950.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
5540269,28,2,7894,6.0,4579.5,2289.75,2.0,1.0,2.0,2015,4,1.0,2289.0,0.0,25880.0,21300.5,1.0,5.0,2.666667,2.081666,1.0,5.0,2.0,-1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6529783,32,36,9103,0.0,0.0,0.00,0.0,0.0,0.0,2015,8,0.0,0.0,0.0,500.0,500.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
6529784,32,36,9107,0.0,0.0,0.00,0.0,0.0,0.0,2015,8,0.0,0.0,0.0,300.0,300.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
6529785,32,36,5704,0.0,0.0,0.00,0.0,0.0,0.0,2015,8,0.0,0.0,0.0,1750.0,1750.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
6529786,32,36,12733,0.0,0.0,0.00,0.0,0.0,0.0,2015,8,0.0,0.0,0.0,2980.0,2980.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0


Mean encoding

In [ ]:
#shop mean encoding
gp_shop_mean = train_set.groupby(['shop_id']).agg({'item_cnt_nextmonth': ['mean']})
gp_shop_mean.columns = ['shop_mean']
gp_shop_mean.reset_index(inplace=True)
#item mean encoding
gp_item_mean = train_set.groupby(['item_id']).agg({'item_cnt_nextmonth': ['mean']})
gp_item_mean.columns = ['item_mean']
gp_item_mean.reset_index(inplace=True)
#shop with item mean encoding
gp_shop_item_mean = train_set.groupby(['item_id', 'shop_id']).agg({'item_cnt_nextmonth':['mean']})
gp_shop_item_mean.columns = ['shop_item_mean']
gp_shop_item_mean.reset_index(inplace=True)
#year mean encoding
gp_year_mean = train_set.groupby(['year']).agg({'item_cnt_nextmonth': ['mean']})
gp_year_mean.columns = ['year_mean']
gp_year_mean.reset_index(inplace=True)
#month mean encoding
gp_month_mean = train_set.groupby(['month']).agg({'item_cnt_nextmonth': ['mean']})
gp_month_mean.columns = ['month_mean']
gp_month_mean.reset_index(inplace=True)

#Add meand encoding features to train set
train_set = pd.merge(train_set, gp_shop_mean, on='shop_id', how='left')
train_set = pd.merge(train_set, gp_item_mean, on='item_id', how='left')
train_set = pd.merge(train_set, gp_shop_item_mean, on=['shop_id', 'item_id'], how='left')
train_set = pd.merge(train_set, gp_year_mean, on='year', how='left')
train_set = pd.merge(train_set, gp_month_mean, on='month', how='left')

#Add meand encoding features to validation set
validation_set = pd.merge(validation_set, gp_shop_mean, on='shop_id', how='left')
validation_set = pd.merge(validation_set, gp_item_mean, on='item_id', how='left')
validation_set = pd.merge(validation_set, gp_shop_item_mean, on=['shop_id', 'item_id'], how='left')
validation_set = pd.merge(validation_set, gp_year_mean, on='year', how='left')
validation_set = pd.merge(validation_set, gp_month_mean, on='month', how='left')

In [ ]:
#Create train and validation sets and labels
X_train = train_set.drop(['item_cnt_nextmonth', 'date_block_num'], axis=1)
Y_train = train_set['item_cnt_nextmonth'].astype(int)
X_validation = validation_set.drop(['item_cnt_nextmonth', 'date_block_num'], axis=1)
Y_validation = validation_set['item_cnt_nextmonth'].astype(int)

Build test set

In [ ]:
test_set

,date_block_num,shop_id,item_id,item_category_id,item_price,mean_item_price,item_cnt,mean_item_cnt,transactions,year,month,item_cnt_nextmonth,item_price_unit,hist_min_item_price,hist_max_item_price,price_increase,item_cnt_min,item_cnt_max,item_cnt_mean,item_cnt_std,item_cnt_shifted1,item_cnt_shifted2,item_cnt_shifted3,item_trend
6529788,33,2,5572,0.0,0.0,0.0,0.0,0.0,0.0,2015,9,NaN,0.0,0.0,18979.5,18979.5,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
6529789,33,2,5643,0.0,0.0,0.0,0.0,0.0,0.0,2015,9,NaN,0.0,0.0,35260.0,35260.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
6529790,33,2,5583,0.0,0.0,0.0,0.0,0.0,0.0,2015,9,NaN,0.0,0.0,5592.0,5592.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
6529791,33,2,7893,0.0,0.0,0.0,0.0,0.0,0.0,2015,9,NaN,0.0,0.0,27950.0,27950.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
6529792,33,2,7894,6.0,9160.0,2290.0,4.0,1.0,4.0,2015,9,NaN,2290.0,0.0,25880.0,16720.0,1.0,4.0,2.0,1.732051,1.0,1.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6727650,33,36,9103,0.0,0.0,0.0,0.0,0.0,0.0,2015,9,NaN,0.0,0.0,500.0,500.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
6727651,33,36,9107,0.0,0.0,0.0,0.0,0.0,0.0,2015,9,NaN,0.0,0.0,300.0,300.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
6727652,33,36,5704,0.0,0.0,0.0,0.0,0.0,0.0,2015,9,NaN,0.0,0.0,1750.0,1750.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
6727653,33,36,12733,0.0,0.0,0.0,0.0,0.0,0.0,2015,9,NaN,0.0,0.0,2980.0,2980.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [ ]:
latest_records = pd.concat([train_set, validation_set]).drop_duplicates(subset=['shop_id', 'item_id'], keep='last')
X_test = pd.merge(test_set, latest_records, on=['shop_id', 'item_id'], how='left', suffixes=['', '_'])
X_test['year'] = 2015
X_test['month'] = 9
X_test.drop('item_cnt_nextmonth', axis=1, inplace=True)
X_test = X_test[X_train.columns]
X_test

,shop_id,item_id,item_category_id,item_price,mean_item_price,item_cnt,mean_item_cnt,transactions,year,month,item_price_unit,hist_min_item_price,hist_max_item_price,price_increase,item_cnt_min,item_cnt_max,item_cnt_mean,item_cnt_std,item_cnt_shifted1,item_cnt_shifted2,item_cnt_shifted3,item_trend,shop_mean,item_mean,shop_item_mean,year_mean,month_mean
0,2,5572,0.0,0.0,0.0,0.0,0.0,0.0,2015,9,0.0,0.0,18979.5,18979.5,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.097169,1.004766,0.84,0.223703,0.165986
1,2,5643,0.0,0.0,0.0,0.0,0.0,0.0,2015,9,0.0,0.0,35260.0,35260.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.097169,1.801337,1.84,0.223703,0.165986
2,2,5583,0.0,0.0,0.0,0.0,0.0,0.0,2015,9,0.0,0.0,5592.0,5592.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.097169,0.589524,0.04,0.223703,0.165986
3,2,7893,0.0,0.0,0.0,0.0,0.0,0.0,2015,9,0.0,0.0,27950.0,27950.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.097169,3.038388,3.12,0.223703,0.165986
4,2,7894,6.0,9160.0,2290.0,4.0,1.0,4.0,2015,9,2290.0,0.0,25880.0,16720.0,1.0,4.0,2.0,1.732051,1.0,1.0,2.0,0.0,0.097169,3.607729,1.68,0.223703,0.165986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197862,36,9103,0.0,0.0,0.0,0.0,0.0,0.0,2015,9,0.0,0.0,500.0,500.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.223703,0.165986
197863,36,9107,0.0,0.0,0.0,0.0,0.0,0.0,2015,9,0.0,0.0,300.0,300.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.223703,0.165986
197864,36,5704,0.0,0.0,0.0,0.0,0.0,0.0,2015,9,0.0,0.0,1750.0,1750.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.223703,0.165986
197865,36,12733,0.0,0.0,0.0,0.0,0.0,0.0,2015,9,0.0,0.0,2980.0,2980.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.223703,0.165986


In [ ]:
#delete unnecessary dataset
del train_set
del validation_set
del test_set

Replacing missing values

In [ ]:
sets = [X_train, X_validation, X_test]

for dataset in sets:
  for shop_id in dataset['shop_id'].unique():
    for column in dataset.columns:
      shop_median = dataset[(dataset['shop_id']==shop_id)][column].median()
      dataset.loc[(dataset[column].isnull()) & (dataset['shop_id']==shop_id), column] = shop_median

X_test.fillna(X_test.mean(), inplace=True)

In [ ]:
X_train.drop(['item_category_id'], axis=1, inplace=True)
X_validation.drop(['item_category_id'], axis=1, inplace=True)
X_test.drop(['item_category_id'], axis=1, inplace=True)

In [ ]:
X_test.head().T

,0,1,2,3,4
shop_id,2.000000,2.000000,2.000000,2.000000,2.000000
item_id,5572.000000,5643.000000,5583.000000,7893.000000,7894.000000
item_price,0.000000,0.000000,0.000000,0.000000,9160.000000
mean_item_price,0.000000,0.000000,0.000000,0.000000,2290.000000
item_cnt,0.000000,0.000000,0.000000,0.000000,4.000000
mean_item_cnt,0.000000,0.000000,0.000000,0.000000,1.000000
transactions,0.000000,0.000000,0.000000,0.000000,4.000000
year,2015.000000,2015.000000,2015.000000,2015.000000,2015.000000
month,9.000000,9.000000,9.000000,9.000000,9.000000
item_price_unit,0.000000,0.000000,0.000000,0.000000,2290.000000


In [ ]:
X_test.describe().T

,count,mean,std,min,25%,50%,75%,max
shop_id,197867.0,3.164284e+01,1.756533e+01,2.000000,16.000000,35.000000,47.000000,5.900000e+01
item_id,197867.0,1.104187e+04,6.210447e+03,30.000000,5383.000000,11266.000000,16068.000000,2.216700e+04
item_price,197867.0,2.913776e+02,2.024288e+03,0.000000,0.000000,0.000000,0.000000,1.821460e+05
mean_item_price,197867.0,1.696889e+02,1.045123e+03,0.000000,0.000000,0.000000,0.000000,4.299000e+04
item_cnt,197867.0,2.568392e-01,9.446527e-01,0.000000,0.000000,0.000000,0.000000,1.900000e+01
mean_item_cnt,197867.0,1.508467e-01,3.825765e-01,0.000000,0.000000,0.000000,0.000000,1.600000e+01
transactions,197867.0,2.360121e-01,7.771042e-01,0.000000,0.000000,0.000000,0.000000,1.500000e+01
year,197867.0,2.015000e+03,0.000000e+00,2015.000000,2015.000000,2015.000000,2015.000000,2.015000e+03
month,197867.0,9.000000e+00,0.000000e+00,9.000000,9.000000,9.000000,9.000000,9.000000e+00
item_price_unit,197867.0,inf,NaN,0.000000,0.000000,0.000000,0.000000,inf
